In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import torch
import torch.nn as nn
import torch.optim as optim
from data_v1 import make_data
from sklearn.preprocessing import StandardScaler
import torch.nn.functional as F
# from data_v2 import make_data

In [3]:
# Define the neural network architecture
class SVM(nn.Module):
    def __init__(self):
        super(SVM, self).__init__()
        self.fc1 = nn.Linear(10, 15)
        self.fc2 = nn.Linear(15, 28)
        # self.fc3 = nn.Linear()
        self.fc4 = nn.Linear(28, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        # x = torch.relu(self.fc3(x))
        x = self.sigmoid(self.fc4(x))
        return x
    


In [4]:

def preprocess_data(batch_size=32, val_split=0.1):
    x_train, y_train, x_test, y_test = make_data()

    # Scale the input data using the standard scaler
    scaler = StandardScaler()
    x_train_normalized = torch.tensor(scaler.fit_transform(x_train), dtype=torch.float)
    x_test_normalized = torch.tensor(scaler.transform(x_test), dtype=torch.float)

    # Split the training data into training and validation sets
    from sklearn.model_selection import train_test_split
    x_train_split, x_val_split, y_train_split, y_val_split = train_test_split(
        x_train_normalized, y_train, test_size=val_split, random_state=42)

    # Combine the input and target data into tuples
    train_data = [(x_train_split[i], y_train_split[i]) for i in range(len(x_train_split))]
    val_data = [(x_val_split[i], y_val_split[i]) for i in range(len(x_val_split))]
    test_data = [(x_test_normalized[i], y_test[i]) for i in range(len(x_test))]

    # Create DataLoader objects for the train, validation, and test data
    trainloader = torch.utils.data.DataLoader(train_data, batch_size=batch_size, shuffle=True)
    valloader = torch.utils.data.DataLoader(val_data, batch_size=batch_size, shuffle=True)
    testloader = torch.utils.data.DataLoader(test_data, batch_size=batch_size, shuffle=True)

    return trainloader, valloader, testloader



In [5]:

epochs=100
lr=0.001
# get the trainloader and testloader


# print(trainloader.dataset[0][0].shape)
# Initialize the network
# net = SVM()
    # net = MLP()

In [34]:

def main():
    
    # Initialize the network
    net = SVM()
    trainloader, validloader, testloader = preprocess_data()

    # net = MLP()

    # Define the loss function and optimizer
    criterion = nn.BCELoss()
    optimizer = optim.Adam(net.parameters(), lr=lr)

    train_losses, valid_losses = [], []
    for epoch in range(epochs):
        running_train_loss, running_valid_loss = 0.0, 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            labels = labels.unsqueeze(1)
            optimizer.zero_grad()
            # print(inputs.shape)

            # Forward pass
            outputs = net(inputs)
            loss = criterion(outputs, labels.float())

            # Backward and optimize
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item()

        # Compute validation loss
        with torch.no_grad():
            for i, data in enumerate(validloader, 0):
                inputs, labels = data
                labels = labels.unsqueeze(1)
                outputs = net(inputs)
                loss = criterion(outputs, labels.float())
                running_valid_loss += loss.item()

        train_loss = running_train_loss / len(trainloader)
        valid_loss = running_valid_loss / len(validloader)

        train_losses.append(train_loss)
        valid_losses.append(valid_loss)

        # print(f"Epoch {epoch+1}, train loss: {train_loss:.3f}, valid loss: {valid_loss:.3f}")



    # Test the model
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            inputs, labels = data
            labels = labels.unsqueeze(1)
            outputs = net(inputs)
            predicted = (outputs >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels.float()).sum().item()

    print(f"Accuracy: {100 * correct/total:.2f}%")

    # # Plot the training and validation loss
    # plt.plot(train_losses, label='Training Loss')
    # plt.plot(valid_losses, label='Validation Loss')
    # plt.legend()
    # plt.xlabel('Epoch')
    # plt.ylabel('Loss')
    # plt.show()
    # print(net)
    return net,100 * correct/total



In [35]:
acc=0
while acc<65:
	model,acc=main()
print(acc)

Accuracy: 50.98%
Accuracy: 53.92%
Accuracy: 57.84%
Accuracy: 47.06%
Accuracy: 54.90%
Accuracy: 50.00%
Accuracy: 50.00%
Accuracy: 55.88%
Accuracy: 39.22%
Accuracy: 56.86%
Accuracy: 49.02%
Accuracy: 50.98%
Accuracy: 67.65%
67.6470588235294


In [36]:
model

SVM(
  (fc1): Linear(in_features=10, out_features=15, bias=True)
  (fc2): Linear(in_features=15, out_features=28, bias=True)
  (fc4): Linear(in_features=28, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)

In [37]:
torch.save(model.state_dict(), 'model_weights.pth')

In [38]:
#load the model_weights.pth to model
model.load_state_dict(torch.load('model_weights.pth'))

<All keys matched successfully>

In [48]:
# print weights of the model layer by layer
# for param_tensor in model.state_dict():
#     print(param_tensor, "\t", model.state_dict()[param_tensor].size())
    
# print the weights of the first layer
print(model.state_dict()['fc1.weight'])

# print the weights of the second layer
print(model.state_dict()['fc2.weight'])

# print the weights of the third layer
# print(model.state_dict()['fc3.weight'])

# print the weights of the fourth layer
print(model.state_dict()['fc4.weight'])

# print the weights of the fifth layer
# print(model.state_dict()['fc5.weight'])



tensor([[ 1.9516e-02, -3.3753e-01,  5.8118e-01,  2.0415e-02,  1.4470e-02,
         -3.8823e-01, -1.1727e-01,  6.1753e-02, -2.7958e-01,  4.9132e-01],
        [-1.7975e-01,  3.7013e-01,  4.2969e-01,  1.7226e-01,  3.1475e-01,
         -2.7447e-01, -3.7376e-02,  3.2705e-01,  1.2583e-01, -3.9519e-01],
        [-3.9365e-01,  3.3264e-01, -5.5171e-01, -2.4816e-01, -2.1076e-01,
         -2.0186e-01,  2.2691e-01, -1.7791e-01, -2.4108e-02, -1.9747e-01],
        [-2.7754e-01, -3.0423e-01, -4.0497e-01,  2.6573e-02,  6.8814e-02,
         -5.5686e-01, -3.7652e-01, -1.2137e-01, -1.6779e-01, -4.0862e-01],
        [ 3.3685e-01, -1.5504e-02, -2.3073e-01,  1.6293e-01, -2.8569e-01,
          3.1163e-01, -3.4972e-01, -9.9787e-02,  1.0516e-01, -4.2642e-01],
        [-2.2154e-01,  3.3974e-02,  1.2887e-01,  2.2012e-01, -2.3108e-01,
         -3.6749e-01, -1.1576e-01, -7.3861e-01,  1.5479e-01,  2.7720e-01],
        [ 2.4090e-01,  1.8826e-01, -6.4668e-02, -1.4563e-01,  3.7702e-01,
         -3.8505e-01,  2.4096e-0

In [54]:
# from the weights of the fc1 layer tell which features are important
weight=model.state_dict()['fc1.weight']
print("fc1")
fc1=[]
# for each of the weight in weight print the index with the highest value
for i in range(len(weight)):
    # print(torch.argmax(weight[i]))
    fc1.append(torch.argmax(weight[i]))

    
# from the weights of the fc2 layer tell which features are important
weight=model.state_dict()['fc2.weight']
fc2=[]
print("fc2")
# for each of the weight in weight print the index with the highest value
for i in range(len(weight)):
    # print(torch.argmax(weight[i]))
	fc2.append(torch.argmax(weight[i]))
    

fc1
fc2


In [67]:
from collections import Counter
# frequency of each feature in fc1
print("fc1")
# for i in fc1:
#     print(i.item(),end=" ")
print(Counter([i.item() for i in fc1]))


fc1
Counter({3: 4, 2: 2, 4: 2, 0: 2, 9: 2, 1: 1, 7: 1, 6: 1})


In [68]:
# frequency of each feature in fc2

print("fc2")
print(Counter([i.item() for i in fc2]))

fc2
Counter({14: 6, 4: 5, 2: 3, 6: 3, 10: 2, 3: 2, 1: 2, 13: 2, 12: 1, 0: 1, 7: 1})
